# Natural Language Processing Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

/home/thojo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7179380916910134923
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1275789312
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12969340071826400143
physical_device_desc: "device: 0, name: Quadro K620, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level=False)
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk

    return None, None
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3

    input_seq = Input(input_shape[1:])
    rnn = GRU(128, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

In [12]:
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 51s 460us/step - loss: 1.7853 - acc: 0.5743 - val_loss: nan - val_acc: 0.6287
Epoch 2/10
110288/110288 [==============================] - 50s 451us/step - loss: 1.2166 - acc: 0.6499 - val_loss: nan - val_acc: 0.6633
Epoch 3/10
110288/110288 [==============================] - 50s 452us/step - loss: 1.0790 - acc: 0.6687 - val_loss: nan - val_acc: 0.6744
Epoch 4/10
110288/110288 [==============================] - 50s 452us/step - loss: 1.0021 - acc: 0.6811 - val_loss: nan - val_acc: 0.6879
Epoch 5/10
110288/110288 [==============================] - 50s 455us/step - loss: 0.9485 - acc: 0.6911 - val_loss: nan - val_acc: 0.6973
Epoch 6/10
110288/110288 [==============================] - 50s 453us/step - loss: 0.9071 - acc: 0.7006 - val_loss: nan - val_acc: 0.7053
Epoch 7/10
110288/110288 [==============================] - 50s 453us/step - loss: 0.8721 - acc: 0.7130 - val_loss: nan -

In [13]:
simple_rnn_model.save('saved_models/simple_rnn_model.h5')
del simple_rnn_model

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [14]:
from keras.layers import Embedding, LSTM

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement model with embedding
    embed_size = 200 
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    embed = Embedding(english_vocab_size, embed_size, input_length=input_shape[-1])(input_seq)
    rnn = LSTM(128, return_sequences=True)(embed) # accuracy = 87%
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)



# TODO: Reshape the input. I'm just padding
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
rnn_with_embedding_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

In [15]:
rnn_with_embedding_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(rnn_with_embedding_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 61s 556us/step - loss: 1.4955 - acc: 0.6638 - val_loss: nan - val_acc: 0.8216
Epoch 2/10
110288/110288 [==============================] - 61s 549us/step - loss: 0.4788 - acc: 0.8562 - val_loss: nan - val_acc: 0.8815
Epoch 3/10
110288/110288 [==============================] - 61s 550us/step - loss: 0.3376 - acc: 0.8943 - val_loss: nan - val_acc: 0.9043
Epoch 4/10
110288/110288 [==============================] - 61s 550us/step - loss: 0.2808 - acc: 0.9103 - val_loss: nan - val_acc: 0.9160
Epoch 5/10
110288/110288 [==============================] - 61s 549us/step - loss: 0.2486 - acc: 0.9191 - val_loss: nan - val_acc: 0.9194
Epoch 6/10
110288/110288 [==============================] - 61s 550us/step - loss: 0.2309 - acc: 0.9236 - val_loss: nan - val_acc: 0.9259
Epoch 7/10
110288/110288 [==============================] - 60s 542us/step - loss: 0.2174 - acc: 0.9272 - val_loss: nan -

In [16]:
rnn_with_embedding_model.save('saved_models/rnn_with_embedding_model.h5')
del rnn_with_embedding_model

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [17]:
from keras.layers import Bidirectional 
from  keras.models  import Sequential
from keras.layers import LSTM

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    hidden_units = 128
    
    # GRU implementation (about 76% accuracy)
    input_seq = Input(input_shape[1:])
    bd = Bidirectional( GRU( hidden_units, return_sequences=True) )( input_seq )
    bd = Bidirectional( GRU( hidden_units, return_sequences=True) )( bd ) #this second layer is just for experimenting
    logits = TimeDistributed(Dense(french_vocab_size))(bd)
    model = Model(input_seq, Activation('softmax')(logits))

    
    # LSTM implementation using Sequential (about 60% accuracy)
#     model = Sequential()
#     model.add(Bidirectional(LSTM(hidden_units, return_sequences = True, dropout = 0.2, recurrent_dropout = 0.2),
#                             input_shape = input_shape[1:]))
#     model.add(Bidirectional(LSTM(hidden_units, return_sequences = True, dropout = 0.2, recurrent_dropout = 0.2) ))
#     model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    ################
    learning_rate = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bidirectional_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

In [18]:
bidirectional_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 137s 1ms/step - loss: 1.3221 - acc: 0.6466 - val_loss: nan - val_acc: 0.7131
Epoch 2/10
110288/110288 [==============================] - 135s 1ms/step - loss: 0.8104 - acc: 0.7426 - val_loss: nan - val_acc: 0.7605
Epoch 3/10
110288/110288 [==============================] - 135s 1ms/step - loss: 0.6788 - acc: 0.7760 - val_loss: nan - val_acc: 0.7765
Epoch 4/10
110288/110288 [==============================] - 134s 1ms/step - loss: 0.5901 - acc: 0.8003 - val_loss: nan - val_acc: 0.8115
Epoch 5/10
110288/110288 [==============================] - 135s 1ms/step - loss: 0.5373 - acc: 0.8155 - val_loss: nan - val_acc: 0.8201
Epoch 6/10
110288/110288 [==============================] - 135s 1ms/step - loss: 0.5044 - acc: 0.8249 - val_loss: nan - val_acc: 0.8073
Epoch 7/10
110288/110288 [==============================] - 137s 1ms/step - loss: 0.4583 - acc: 0.8388 - val_loss: nan - val_ac

In [19]:
bidirectional_model.save('saved_models/bidirectional_model.h5')
del bidirectional_model

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [41]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    num_hidden = 128
#     ###################################### USING LSTMs ####### accuracy about 70%
#     model = Sequential()
#     # Encoder
#     model.add(LSTM(num_hidden, go_backwards=True, input_shape=input_shape[1:])) # ?
#     model.add(RepeatVector(output_sequence_length))
#     # Decoder
#     model.add(LSTM(num_hidden, return_sequences=True))
#     model.add(TimeDistributed(Dense(french_vocab_size)))
#     model.add(Activation('softmax'))    

    ##################################### USING GRUs ####### accuracy about 75%
    model = Sequential()
    # Encoder
    model.add(GRU(num_hidden, go_backwards=True, input_shape=input_shape[1:]))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(GRU(num_hidden, return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size)))
    model.add(Activation('softmax'))    

    
    ##########################
  
    # Compile model
    learning_rate = 1e-3
    model.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = Adam(learning_rate),
                 metrics = ['accuracy'])
    return model
tests.test_encdec_model(encdec_model)
# OPTIONAL: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# Train the neural network
ed_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(ed_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_20 (GRU)                 (None, 128)               49920     
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 21, 128)           0         
_________________________________________________________________
gru_21 (GRU)                 (None, 21, 128)           98688     
_________________________________________________________________
time_distributed_22 (TimeDis (None, 21, 344)           44376     
_________________________________________________________________
activation_18 (Activation)   (None, 21, 344)           0         
Total params: 192,984
Trainable params: 192,984
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
ed_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text( ed_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 87s 789us/step - loss: 2.0255 - acc: 0.5288 - val_loss: nan - val_acc: 0.5953
Epoch 2/10
110288/110288 [==============================] - 82s 739us/step - loss: 1.4069 - acc: 0.6172 - val_loss: nan - val_acc: 0.6350
Epoch 3/10
110288/110288 [==============================] - 81s 736us/step - loss: 1.2016 - acc: 0.6577 - val_loss: nan - val_acc: 0.6762
Epoch 4/10
110288/110288 [==============================] - 82s 741us/step - loss: 1.0787 - acc: 0.6800 - val_loss: nan - val_acc: 0.6854
Epoch 5/10
110288/110288 [==============================] - 83s 750us/step - loss: 1.0334 - acc: 0.6864 - val_loss: nan - val_acc: 0.6799
Epoch 6/10
110288/110288 [==============================] - 83s 751us/step - loss: 0.9785 - acc: 0.7007 - val_loss: nan - val_acc: 0.6981
Epoch 7/10
110288/110288 [==============================] - 82s 745us/step - loss: 0.9265 - acc: 0.7144 - val_loss: nan -

In [43]:
ed_model.save('saved_models/ed_model.h5')
del ed_model

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [23]:
from keras.layers import Dropout
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    embed_size = 200    
    learning_rate = 1e-3
    num_hidden = 128
    
    model = Sequential()
    # Encoder part
    model.add( Embedding(english_vocab_size, embed_size, input_length=input_shape[-1] ) )    
    model.add( Bidirectional( LSTM(num_hidden, go_backwards=True, input_shape=input_shape[1:],  dropout = 0.4, recurrent_dropout = 0.4)) )  #keep the go_backwards?
    model.add( RepeatVector(output_sequence_length) )
    # Decoder part
    model.add( Bidirectional( LSTM(num_hidden, return_sequences=True, dropout = 0.4, recurrent_dropout = 0.4)) ) 
    model.add( TimeDistributed( Dense(512) ) )
    model.add(Dropout(0.5))
    model.add( TimeDistributed( Dense(french_vocab_size) ) )
    model.add( Activation('softmax') )

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
    
    return None
tests.test_model_final(model_final)
print('Final Model Loaded')

Final Model Loaded


In [24]:
# TODO: Train the final model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


final_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(final_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1, 200)            39800     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 256)               336896    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 21, 256)           394240    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 21, 512)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 21, 344)           176472    
__________

## Prediction (IMPLEMENTATION)

In [25]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # note: x and y are already pre-processed
    # TODO: Train neural network using model_final
    model = model_final(x.shape, 
                        y.shape[1], 
                        len(x_tk.word_index), 
                        len(y_tk.word_index))
    
    model.fit(x, y, batch_size=128, epochs=10, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))
    return model

In [26]:
full_model=final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 192s 2ms/step - loss: 1.7173 - acc: 0.5724 - val_loss: nan - val_acc: 0.7043
Epoch 2/10
110288/110288 [==============================] - 180s 2ms/step - loss: 1.0120 - acc: 0.7063 - val_loss: nan - val_acc: 0.7897
Epoch 3/10
110288/110288 [==============================] - 180s 2ms/step - loss: 0.7296 - acc: 0.7769 - val_loss: nan - val_acc: 0.8621
Epoch 4/10
110288/110288 [==============================] - 180s 2ms/step - loss: 0.5370 - acc: 0.8330 - val_loss: nan - val_acc: 0.9133
Epoch 5/10
110288/110288 [==============================] - 180s 2ms/step - loss: 0.4082 - acc: 0.8741 - val_loss: nan - val_acc: 0.9344
Epoch 6/10
110288/110288 [==============================] - 180s 2ms/step - loss: 0.3376 - acc: 0.8968 - val_loss: nan - val_acc: 0.9424
Epoch 7/10
110288/110288 [==============================] - 180s 2ms/step - loss: 0.2945 - acc: 0.9105 - val_loss: nan - val_ac

In [27]:
full_model.save('saved_models/full_model.h5')
del full_model

# project notes:

-  batch_size = 1024 works fine (though slow in all the above).  However, when I did the test-train split below, the GPu on my machine does not have enough memory for that batch_size for the full_model.  Therefore, for an apples-to-apples comparison, I have reduced the batch_size to 128 for all models in this notebook.

-  I have used `go_backwards=True` for the encoding RNNs in the Encoder-Decoder model as well as the final_model per suggestions in places such as __[this link](https://cntk.ai/pythondocs/CNTK_204_Sequence_To_Sequence.html)__ which advice to use `False` for the attention models and `True` otherwise. However, I found that the models here also work with the default `go_backwards=False` but just have lower accuracy by about 5%. 

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( english_sentences, french_sentences, test_size=0.1, random_state=42)

tt_preproc_english_sentences, tt_preproc_french_sentences, tt_english_tokenizer, tt_french_tokenizer =\
    preprocess(X_train, y_train)
    
tt_max_english_sequence_length = tt_preproc_english_sentences.shape[1]
tt_max_french_sequence_length = tt_preproc_french_sentences.shape[1]
tt_english_vocab_size = len(tt_english_tokenizer.word_index)
tt_french_vocab_size = len(tt_french_tokenizer.word_index)

tt_x_pad = pad(tt_preproc_english_sentences, tt_max_french_sequence_length)
tt_x = tt_x_pad.reshape((-1, tt_preproc_french_sentences.shape[-2], 1))

test_preproc_english_sentences, test_preproc_french_sentences, _, _ = preprocess( X_test, y_test )
test_x_pad = pad(test_preproc_english_sentences, tt_preproc_french_sentences.shape[1])
test_x = test_x_pad.reshape((-1, tt_preproc_french_sentences.shape[1], 1))
test_y = pad(test_preproc_french_sentences, tt_preproc_french_sentences.shape[1]) 

print('Data Preprocessed')
print("Max English sentence length:", tt_max_english_sequence_length)
print("Max French sentence length:", tt_max_french_sequence_length)
print("English vocabulary size:", tt_english_vocab_size)
print("French vocabulary size:", tt_french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [45]:
# Train the simple neural network on the training sample alone
tt_simple_rnn_model = simple_model(    
    tt_x.shape,
    tt_max_french_sequence_length,
    tt_english_vocab_size,
    tt_french_vocab_size)
tt_simple_rnn_model.fit(tt_x, tt_preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.1) 
print(logits_to_text(tt_simple_rnn_model.predict(tt_x[:1])[0], tt_french_tokenizer))

Train on 111666 samples, validate on 12408 samples
Epoch 1/10
111666/111666 [==============================] - 53s 477us/step - loss: 1.7669 - acc: 0.5773 - val_loss: nan - val_acc: 0.6452
Epoch 2/10
111666/111666 [==============================] - 49s 441us/step - loss: 1.2074 - acc: 0.6598 - val_loss: nan - val_acc: 0.6728
Epoch 3/10
111666/111666 [==============================] - 48s 426us/step - loss: 1.0708 - acc: 0.6814 - val_loss: nan - val_acc: 0.6823
Epoch 4/10
111666/111666 [==============================] - 48s 426us/step - loss: 0.9932 - acc: 0.6963 - val_loss: nan - val_acc: 0.7023
Epoch 5/10
111666/111666 [==============================] - 48s 427us/step - loss: 0.9364 - acc: 0.7121 - val_loss: nan - val_acc: 0.7197
Epoch 6/10
111666/111666 [==============================] - 49s 443us/step - loss: 0.8927 - acc: 0.7245 - val_loss: nan - val_acc: 0.7354
Epoch 7/10
111666/111666 [==============================] - 48s 433us/step - loss: 0.8615 - acc: 0.7336 - val_loss: nan -

In [30]:
# #code sample to load and test a model
# from keras.models import load_model
# simple_rnn_model = load_model('saved_models/simple_rnn_model.h5')
# print(logits_to_text(simple_rnn_model.predict(tmp_x[:10])[0], french_tokenizer))

In [46]:
score, acc = tt_simple_rnn_model.evaluate(test_x, test_y, batch_size=128)
print( 'simple model true score = {:4.2f}'.format(score) )
print( 'simple model true acc = {:4.2f}'.format(acc) )

13787/13787 [==============================] - 2s 159us/step
simple model true score = 3.14
simple model true acc = 0.57


In [47]:
tt_simple_rrn_model.save('saved_models/tt_simple_rrn_model.h5')
del tt_simple_rnn_model

The accuracy for the simple RNN model dropped to 0.57 from 0.76 when proper test-train split was done.

# (optional continued) Next we look at proper accuracy evaluated for a bidirectional RNN model

In [32]:
tt_ed_model = encdec_model(
    tt_x.shape,
    tt_max_french_sequence_length,
    tt_english_vocab_size,
    tt_french_vocab_size)
tt_ed_model.fit(tt_x, tt_preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.1) 
print(logits_to_text(tt_ed_model.predict(tt_x[:1])[0], tt_french_tokenizer))

Train on 111666 samples, validate on 12408 samples
Epoch 1/10
111666/111666 [==============================] - 81s 727us/step - loss: 1.9292 - acc: 0.5447 - val_loss: nan - val_acc: 0.6046
Epoch 2/10
111666/111666 [==============================] - 80s 719us/step - loss: 1.3061 - acc: 0.6411 - val_loss: nan - val_acc: 0.6621
Epoch 3/10
111666/111666 [==============================] - 80s 720us/step - loss: 1.1491 - acc: 0.6722 - val_loss: nan - val_acc: 0.6782
Epoch 4/10
111666/111666 [==============================] - 79s 708us/step - loss: 1.0619 - acc: 0.6876 - val_loss: nan - val_acc: 0.6928
Epoch 5/10
111666/111666 [==============================] - 79s 706us/step - loss: 0.9815 - acc: 0.7035 - val_loss: nan - val_acc: 0.7153
Epoch 6/10
111666/111666 [==============================] - 79s 705us/step - loss: 0.8938 - acc: 0.7282 - val_loss: nan - val_acc: 0.7357
Epoch 7/10
111666/111666 [==============================] - 79s 706us/step - loss: 0.8256 - acc: 0.7418 - val_loss: nan -

In [33]:
score, acc = tt_ed_model.evaluate(test_x, test_y, batch_size=1024)
print( 'Encoder-decoder model true score = {:4.2f}'.format(score) )
print( 'Encoder-decoder model acc = {:4.2f}'.format(acc) )

13787/13787 [==============================] - 1s 102us/step
Encoder-decoder model true score = 3.24
Encoder-decoder model acc = 0.56


In [ ]:
tt_ed_model.save('saved_models/tt_ed_model.h5')
del tt_ed_model

The accuracy for the Encoder-decoder model dropped to 0.56 from 0.75 when proper test-train split was done.

# (optional continued) Next we look at proper accuracy evaluated for the final model

In [48]:
tt_final_model = model_final(
    tt_x_pad.shape,
    tt_max_french_sequence_length,
    tt_english_vocab_size,
    tt_french_vocab_size)
tt_final_model.fit(tt_x_pad, tt_preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.1) 

Train on 111666 samples, validate on 12408 samples
Epoch 1/10
111666/111666 [==============================] - 205s 2ms/step - loss: 1.7198 - acc: 0.5724 - val_loss: nan - val_acc: 0.6990
Epoch 2/10
111666/111666 [==============================] - 199s 2ms/step - loss: 1.0380 - acc: 0.7013 - val_loss: nan - val_acc: 0.7782
Epoch 3/10
111666/111666 [==============================] - 200s 2ms/step - loss: 0.7789 - acc: 0.7646 - val_loss: nan - val_acc: 0.8405
Epoch 4/10
111666/111666 [==============================] - 206s 2ms/step - loss: 0.5930 - acc: 0.8160 - val_loss: nan - val_acc: 0.8932
Epoch 5/10
111666/111666 [==============================] - 201s 2ms/step - loss: 0.4646 - acc: 0.8565 - val_loss: nan - val_acc: 0.9230
Epoch 6/10
111666/111666 [==============================] - 197s 2ms/step - loss: 0.3803 - acc: 0.8838 - val_loss: nan - val_acc: 0.9362
Epoch 7/10
111666/111666 [==============================] - 197s 2ms/step - loss: 0.3267 - acc: 0.9002 - val_loss: nan - val_ac

In [49]:
print(logits_to_text(tt_final_model.predict(tt_x_pad[:1])[0], tt_french_tokenizer))

la pomme est son fruit préféré mais le citron est votre favori <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [52]:
score, acc = tt_final_model.evaluate(test_x_pad, test_y, batch_size=128)
print( 'Final model true score = {:4.2f}'.format(score) )
print( 'Final model acc = {:4.2f}'.format(acc) )

13787/13787 [==============================] - 7s 502us/step
Final model true score = 4.98
Final model acc = 0.53


In [53]:
tt_final_model.save('saved_models/tt_final_model.h5')
del tt_final_model

The accuracy for the final model dropped to 0.53 from 0.96 when proper test-train split was done.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [38]:
# Save before you run this cell!
# !!jupyter nbconvert *.ipynb